In [1]:
require('Matrix')
require('gurobi')
thePath = "C:/Users/Isabelle_Salcher/OneDrive/NYU/03 - Year 2/Match Econ Code/04-appli-optassign"

Loading required package: Matrix
Loading required package: gurobi
Loading required package: slam


In [2]:
data = as.matrix(read.csv(paste0(thePath,"/affinitymatrix.csv"),sep=",", header=TRUE)) # loads the data
nbcar = 10
A = matrix(as.numeric(data[1:nbcar,2:(nbcar+1)]),nbcar,nbcar)

In [4]:
data = as.matrix(read.csv(paste0(thePath,"/Xvals.csv"),sep=",", header=TRUE)) # loads the data
Xvals = matrix(as.numeric(data[,1:nbcar]),ncol=nbcar)

data = as.matrix(read.csv(paste0(thePath,"/Yvals.csv"),sep=",", header=TRUE)) # loads the data
Yvals = matrix(as.numeric(data[,1:nbcar]),ncol=nbcar)

In [6]:
sdX = apply(Xvals,2,sd)
sdY = apply(Yvals,2,sd)
mX = apply(Xvals,2,mean)
mY = apply(Yvals,2,mean)
Xvals = t( (t(Xvals)-mX) / sdX)
Yvals = t( (t(Yvals)-mY) / sdY)
nobs = dim(Xvals)[1]

In [11]:
Phi = Xvals %*% A %*% t(Yvals)

In [15]:
p = rep(1/nobs,nobs)
q = rep(1/nobs,nobs)

In [16]:
N = dim(Phi)[1]
M = dim(Phi)[2]

In [19]:
c = c(Phi)

In [20]:
A1 = kronecker(matrix(1,1,M),sparseMatrix(1:N,1:N))
A2 = kronecker(sparseMatrix(1:M,1:M),matrix(1,1,N))
A = rbind2(A1,A2)

d = c(p,q) 

result   = gurobi ( list(A=A,obj=c,modelsense="max",rhs=d,sense="="), params=list(OutputFlag=0) ) 
if (result$status=="OPTIMAL") {
  pi = matrix(result$x,nrow=N)
  u = result$pi[1:N]
  v = result$pi[(N+1):(N+M)]
  val = result$objval
} else {stop("optimization problem with Gurobi.") }

print(paste0("Value of the problem (Gurobi) =",val))
print(u[1:10])
print(v[1:10])

[1] "Value of the problem (Gurobi) =1.70388302245657"
 [1]  0.51218086 -0.11429331  0.94097649  1.63155380  2.33101773  1.32786946
 [7] -0.15706008  0.59484033  0.07083777  0.12608723
 [1] 0.6680330 0.4496133 2.0290215 1.0974134 0.3045042 1.3137156 0.6013508
 [8] 1.9163097 0.8227658 1.2977535
